# **TEXT CLASSIFICATION CODE** 


In [110]:
import sys

#!{sys.executable} -m pip install numpy
import numpy as np

#!{sys.executable} -m pip install sklearn
from sklearn import metrics

#!{sys.executable} -m pip install pandas
import pandas as pd

#!{sys.executable} -m pip install nltk
import nltk

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#!{sys.executable} -m pip install nbformat
!{sys.executable} -m pip install pattern3

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [101]:
%run ./Text_Normalization_Function.ipynb  #defining text normalization function

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


[nltk_data] Downloading package omw-1.4 to /Users/kkee/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-co

[nltk_data] Downloading package stopwords to /Users/kkee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kkee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kkee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/kkee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Let's now define a function and call it **fmat_descr_fun** to be used later to describe the feature matrix (vectorized corpus). The function prints out the dimensions of the matrix, share of non-zero elements and so on. The function will take two inputs: the feature matrix and your vectorizer function. In what follows, you can see the descriptives that the function provides:

In [102]:
def fmat_descr_fun(your_feature_matrix,your_vectorizer):
    print("Dimensions (number of posts, number of features): ", your_feature_matrix.shape)  
    print("The first 5 features - names: ", your_vectorizer.get_feature_names()[0:5]) 
    print("Share of non-zero elements in the matrix: ", 
          your_feature_matrix.nnz / (float(your_feature_matrix.shape[0]) * float(your_feature_matrix.shape[1]))) #nnz: Get the count of explicitly-stored values (nonzeros)
    print("Average number of features present, per post: ", 
          round(your_feature_matrix.nnz/float(your_feature_matrix.shape[0]),1))

# Text classification 

In [131]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.naive_bayes import MultinomialNB

## Seperate chuncks of code

In [143]:
#normalization

def normalize_text(corpus_train, corpus_test):
     normalized_train = normalize_corpus(corpus_train.data)
     normalized_test = normalize_corpus(corpus_test.data)
     
     return normalized_train,normalized_test

In [133]:
#vectorization

def vectorize_text(normalized_train,normalized_test,vectorizer,max_features):
     if vectorizer == 'bow':
          vectorizer = CountVectorizer()
          vectorized_train = vectorizer.fit_transform(normalized_train)
          vectorized_test = vectorizer.transform(normalized_test)
          
     elif vectorizer == 'tf-idf':
          vectorizer = TfidfVectorizer(norm = 'l2', smooth_idf = True)
          vectorized_train = vectorizer.fit_transform(normalized_train).todense()
          vectorized_test = vectorizer.transform(normalized_test).todense()
          
     elif vectorizer == 'n-grams':
          vectorizer = CountVectorizer(max_features=1000, ngram_range=(1, 2))
          vectorized_train = vectorizer.fit_transform(normalized_train).toarray()
          vectorized_test = vectorizer.transform(normalized_test).toarray()
     
     return vectorized_train,vectorized_test

In [156]:
#feature selection

# metric:chi2,mutual_info_classif
def feature_selection(vectorized_train,vectorized_test,metric,best_feature_num):
     kbest = SelectKBest(score_func = metric, k = best_feature_num)
     kbest_train = kbest.fit_transform(vectorized_train, corpus_train.target)
     kbest_test = kbest.transform(vectorized_test)
   
     return kbest_train,kbest_test

In [146]:
#classification model 

def classify_text(model,kbest_train,kbest_test):
     if model == 'MultinomialNB':
          model = MultinomialNB(alpha=0.1) 
     
     model.fit(kbest_train, corpus_train.target)
     predicted = model.predict(kbest_test)

     confusion_matrix = metrics.confusion_matrix(corpus_test.target, predicted)
     confusion_matrix_df = pd.DataFrame(data = confusion_matrix, 
                                        columns = corpus_train.target_names,
                                        index = corpus_train.target_names)
     accuracy = metrics.accuracy_score(corpus_test.target, predicted)
     
     return accuracy,confusion_matrix_df

## summary

In [160]:
def text_classication_4steps(corpus_train,corpus_test, max_features, best_feature_num, vectorizer = 'bow', metric = chi2, model = 'MultinomialNB'):
     #vectorizer:'bow','tf-idf','n-grams'
     #metric:chi2,mutual_info_classif
     #model:'MultinomialNB'
     normalized_train,normalized_test = normalize_text(corpus_train,corpus_test)
     vectorized_train,vectorized_test = vectorize_text(normalized_train,normalized_test,vectorizer,max_features)
     kbest_train,kbest_test = feature_selection(vectorized_train,vectorized_test,metric,best_feature_num)
     accuracy,confusion_matrix_df = classify_text(model,kbest_train,kbest_test)
     
     return accuracy,confusion_matrix_df

In [162]:
# text_classication_4steps(corpus_train,corpus_test,
#                          max_features = 1000,
#                          best_feature_num = 800,
#                          vectorizer = 'tf-idf',  #vectorizer:'bow','tf-idf','n-grams'
#                          metric = chi2, #metric:chi2,mutual_info_classif
#                          model = 'MultinomialNB') #model:'MultinomialNB'

(0.7723577235772358,
                     alt.atheism  comp.graphics  sci.space  talk.religion.misc
 alt.atheism                 211             13         49                  46
 comp.graphics                 4            356         28                   1
 sci.space                    17             23        352                   2
 talk.religion.misc           79             11         35                 126)

In [168]:
def text_classication_3steps(normalized_train,normalized_test, max_features, best_feature_num, vectorizer = 'bow', metric = chi2, model = 'MultinomialNB'):
     #vectorizer:'bow','tf-idf','n-grams'
     #metric:chi2,mutual_info_classif
     #model:'MultinomialNB'
     vectorized_train,vectorized_test = vectorize_text(normalized_train,normalized_test,vectorizer,max_features)
     kbest_train,kbest_test = feature_selection(vectorized_train,vectorized_test,metric,best_feature_num)
     accuracy,confusion_matrix_df = classify_text(model,kbest_train,kbest_test)
     
     return accuracy,confusion_matrix_df

In [173]:
# normalized_train,normalized_test = normalize_text(corpus_train, corpus_test)
# text_classication_3steps(normalized_train,normalized_test,
#                          max_features = 1000,
#                          best_feature_num = 800,
#                          vectorizer = 'tf-idf',  #vectorizer:'bow','tf-idf','n-grams'
#                          metric = chi2, #metric:chi2,mutual_info_classif
#                          model = 'MultinomialNB') #model:'MultinomialNB'

(0.7723577235772358,
                     alt.atheism  comp.graphics  sci.space  talk.religion.misc
 alt.atheism                 211             13         49                  46
 comp.graphics                 4            356         28                   1
 sci.space                    17             23        352                   2
 talk.religion.misc           79             11         35                 126)

## sample data

In [157]:
# from sklearn.datasets import fetch_20newsgroups
# warnings.filterwarnings("ignore", category=DeprecationWarning) #supress deprication warnings if any

# categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# #Out of 20 newsgroups (topics) available, we will use posts on **4** topics: atheism, religion, computer graphics, and science. You can refer to those newsgroups as **classes** or **categories**. Note right away, that the "atheism" and "religion" classes are likely **similar** to each other: people might be using similar words when they talk about atheism and religion.

# corpus_train = fetch_20newsgroups(categories = categories,
#                                   subset = 'train', 
#                                   remove = ('headers', 'footers', 'quotes')) 

# corpus_test = fetch_20newsgroups(categories = categories,
#                                  subset='test', 
#                                  remove=('headers', 'footers', 'quotes')) 

# corpus_train_data = corpus_train.data
# corpus_test_data = corpus_test.data
# train_target = corpus_train.target
# test_target = corpus_test.target
# train_target_names = corpus_train.target_names
# print(corpus_train.data[7])     

In [ ]:
1